In [60]:
import requests
import pandas as pd
import numpy as np
import os
from datetime import datetime, date, timedelta
from dateutil import relativedelta
import time
import pyarrow as pa
from urllib.parse import quote_plus
from pandas.api.types import is_string_dtype
import gc
import warnings

import config
from normalize_funcs import *
from db_funcs import createDBTable, downloadTableToDB, get_mssql_table, removeRowsFromDB

start_of_the_time = datetime.strptime('1990-01-01', '%Y-%m-%d') # указываем дату начала сбора данных, для преобразования номера месяца

start_date = '2023-01-01'
start_date = datetime.strptime(start_date, '%Y-%m-%d').date()

end_date = '2023-01-02'
end_date = datetime.strptime(end_date, '%Y-%m-%d').date()

print(f'start_date: {start_date} / end_date: {end_date}')




start_date: 2023-01-01 / end_date: 2023-01-02


In [34]:
# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем ширину столбцов по контенту
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

warnings.simplefilter(action='ignore', category=FutureWarning)
# убираем лишние предупреждения
pd.set_option('mode.chained_assignment', None)

sep_str = '*' * 50

In [58]:
# создаем функцию, которая вернет номер месяца от начальной даты отсчета
def get_mon_num_from_date(curr_date, start_of_the_time):
    # с помощью метода relativedelta получаем кол-во лет и месяцев от даты начала до текущей даты
    relative_date = relativedelta.relativedelta(curr_date, start_of_the_time)
    # теперь нужно получить номер текущего месяца от даты начала
    months_count = relative_date.years * 12 + relative_date.months
    
    return months_count

In [61]:
print(get_mon_num_from_date(start_date, start_of_the_time))

396


In [35]:
query = f"select * from tv_Ad where vid='4404947'"

In [36]:
df = get_mssql_table(config.investments_db_name, query=query, conn_lst=config.conn_lst)

Все ок. Подключились!
Загрузка завершена успешно


In [37]:
df

,vid,name,ename,notes,atid,stdur,alid,blid,mlid,slid2,slid3,slid4,fiss,sblid,slaid,slvid,stid,alidm,blidm,mlidm,slid2m,slid3m,slid4m,sblidm,agrid,qr,qr_dur,qr_url
0,4404947,"WILDBERRIES - (ADIDAS , 55%) www","WILDBERRIES internet-shop (ADIDAS sports clothing, discount to 55%) www","., www, , ., , 4 ., , ., www",1,10,565082,2046493,4919449,50125,150232,421513,20230102,1115769,1,1,9,565082,2046493,4919449,50125,150232,421513,1115769,0,N,0,


In [38]:
query = f"select * from BrandList where blid='2046493'"
df = get_mssql_table(config.investments_db_name, query=query, conn_lst=config.conn_lst)

Все ок. Подключились!
Загрузка завершена успешно


In [39]:
df

,blid,name,ename
0,2046493,ADIDAS; WILDBERRIES,ADIDAS; WILDBERRIES


In [40]:
query = f"select * from tv_Appendix where vid='4404947'"
df = get_mssql_table(config.investments_db_name, query=query, conn_lst=config.conn_lst)

Все ок. Подключились!
Загрузка завершена успешно


In [41]:
df

,vid,mid,aid,sid2,sid3,sid4,sbid,bid,main
0,4404947,29674,428800,59,4369,684,3770,189,Y
1,4404947,2716138,428800,2587,4892,4977,556193,568656,Y


In [42]:
query = f"select * from Brand where bid='568656'"
df = get_mssql_table(config.investments_db_name, query=query, conn_lst=config.conn_lst)

Все ок. Подключились!
Загрузка завершена успешно


In [43]:
df

,bid,name,ename,notes
0,568656,WILDBERRIES,WILDBERRIES,-


In [67]:
query = f"""select 
* 
from tv_Ad_month t1 
where t1.mon='396' and t1.vid='4404947'  
"""
df = get_mssql_table(config.investments_db_name, query=query, conn_lst=config.conn_lst)

Все ок. Подключились!
Загрузка завершена успешно


In [68]:
df.shape

(448, 20)

In [ ]:
left join tv_Appendix t2 
on t1.vid=t2.vid
(t2.sid2 in (2408, 4780, 4926, 5028) or t2.sbid in (137166, 489562, 494633, 556193, 872285, 966983, 1155675, 1245528))
and t2.bid='568656'

In [79]:
query = f"""select 
t1.vid, t2.bid, t3.blid
from tv_Ad_month t1 left join tv_Appendix t2 
on t1.vid=t2.vid
left join tv_Ad t3
on t1.vid=t3.vid
where t1.mon='396' and t1.vid='4404947'  and 
(t2.sid2 in (2408, 4780, 4926, 5028) or t2.sbid in (137166, 489562, 494633, 556193, 872285, 966983, 1155675, 1245528))
"""
df = get_mssql_table(config.investments_db_name, query=query, conn_lst=config.conn_lst)

Все ок. Подключились!
Загрузка завершена успешно


In [80]:
df.shape

(448, 3)

In [81]:
df.head(2)

,vid,bid,blid
0,4404947,568656,2046493
1,4404947,568656,2046493


In [82]:
df['bid'].drop_duplicates()

0    568656
Name: bid, dtype: int64

start_date: 2023-01-01 / end_date: 2023-01-02


360


In [ ]:

segments_main_link = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vR2WJjY1kEsKluiXddRiFrqKFOVGO06eJY4CVt33OhJ7KdALHHr3sYsqXyeoVxmNI08VNUP6mAlnQKt/pub?gid=0&single=true&output=csv'

media_type_detail_link = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vTmkq1SXPqmHqCx9MfiMlbf3nV3PRKctT0r1RItnLxnKt7MYzBg56V99obmqbjWek3ux8gdjhvLQR8E/pub?gid=0&single=true&output=csv'

tv_net_group_link = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vTvelFTRLTMGnlLWLmXvLXRLVdBe7magXnTfncl72nXYN0l6s2NfPLLe-M2ipdpV94JesJ-ZuA6lejj/pub?gid=0&single=true&output=csv'

In [ ]:
df = get_mssql_table(db_name, table_name='', query='')